# 2x2 Plaquette with Hubbard Interaction (Wide-Band Limit)

We calculate the Green function of a 2x2 plaquette with Hubbard interaction coupled to a featureless bath (wide-band limit).

The plaquette has 4 sites arranged as:
```
0 - 1
|   |
2 - 3
```
with periodic boundary conditions (hopping 0↔1, 1↔3, 3↔2, 2↔0).

The non-interacting Hamiltonian (per spin) is given by the hopping matrix:

$$
\hat{h}_0 = -\begin{pmatrix}
\mu & t & t & 0 \\
t & \mu & 0 & t \\
t & 0 & \mu & t \\
0 & t & t & \mu
\end{pmatrix}
$$

The interaction is of the single-band Hubbard type on each site:

$$h_{\rm int} = U \sum_{i=0}^{3} n_{\uparrow,i} n_{\downarrow,i}$$

The hybridization function in the wide-band limit is:

$$\Delta(i\omega_n) = -i \Gamma \, {\rm sgn}(\omega_n) \, \mathbb{1}_{4\times 4}$$

The parameters are defined below.

In [9]:
# %load model.py
import sys, os
sys.path.append(os.getcwd() + '/../common')
from util import *

from triqs.gf import Gf, MeshImFreq, BlockGf, iOmega_n, inverse
from triqs.operators import c, c_dag, n
from numpy import array, sign, eye, matrix

# ==== System Parameters ====
beta = 25.0                     # Inverse temperature
U = 2.0                         # Hubbard interaction
t = 1.0                         # Hopping
mu = U / 2                      # Chemical potential (half-filling)
Gamma = 0.1                     # Hybridization strength (wide-band limit)

n_orb = 4                       # 4 sites in 2x2 plaquette
n_iw = int(10 * beta)           # Matsubara frequencies

block_names = ['up', 'dn']

# ==== Hopping matrix (2x2 plaquette with PBC) ====
# Site numbering: 0-1
#                 2-3
# Hopping: 0↔1, 1↔3, 3↔2, 2↔0
h_0_mat = -array([
    [mu,  t, t, 0],
    [ t, mu, 0, t],
    [ t,  0, mu, t],
    [ 0,  t, t, mu],
])

# ==== Interaction Hamiltonian ====
h_int = sum(U * n('up', i) * n('dn', i) for i in range(n_orb))

# ==== Local Hamiltonian (quadratic part) ====
c_dag_vec = {s: matrix([[c_dag(s, o) for o in range(n_orb)]]) for s in block_names}
c_vec = {s: matrix([[c(s, o)] for o in range(n_orb)]) for s in block_names}
h_0 = sum(c_dag_vec[s] * h_0_mat * c_vec[s] for s in block_names)[0, 0]

h_imp = h_0 + h_int

# ==== Green function structure ====
gf_struct = [(s, n_orb) for s in block_names]

# ==== Hybridization Function (wide-band limit: -i*Gamma*sign(w)) ====
iw_mesh = MeshImFreq(beta, 'Fermion', n_iw)
Delta_iw = BlockGf(mesh=iw_mesh, gf_struct=gf_struct)
for bl in block_names:
    for iw in Delta_iw[bl].mesh:
        Delta_iw[bl][iw] = -1j * Gamma * sign(iw.imag) * eye(n_orb)

# ==== Non-Interacting Green function ====
G0_iw = BlockGf(mesh=iw_mesh, gf_struct=gf_struct)
for bl in block_names:
    G0_iw[bl] << inverse(iOmega_n - h_0_mat - Delta_iw[bl])


## Results

In [6]:
from triqs.plot.mpl_interface import oplot, plt
%matplotlib inline

In [8]:
# %load ../common/plot.py
import sys, os
sys.path.append(os.getcwd() + "/..")
sys.path.append(os.getcwd() + "/../../common")
from model import *

from h5 import HDFArchive
from triqs.plot.mpl_interface import oplot, plt
from glob import glob
from os.path import basename

# === Load Green function for every solver and calculate self-energy

solver_lst = [ basename(f).strip('.h5') for f in glob('results/*.h5') ]
marker_lst = ['-x', '-+', '-^', '-v', '-<', '->', '-*', '-p']
G, Sigma = {}, {}

for solver in solver_lst:
    dat = HDFArchive('results/' + solver + '.h5','r')
    G[solver] = dat['G']
    Sigma[solver] = G0_iw.copy()
    Sigma[solver] << inverse(G0_iw) - inverse(G[solver])

# === For every block and solver, plot Green function and Self energy

block_lst = list(G[solver_lst[0]].indices)
n_blocks = len(block_lst)

for g, name in [[G, 'G'], [Sigma, r'$\Sigma$']]:

    plt.subplots(n_blocks,1,figsize=(10,6*n_blocks))

    for i, block in enumerate(block_lst,1):
        fig = plt.subplot(n_blocks,1,i)
        fig.set_title(name + "[" + block + "]")
        for solver in solver_lst:
            marker = marker_lst[solver_lst.index(solver)]
            oplot(g[solver][block][0,0], marker, name = name + "[0,0]_%s" % solver)
        plt.xlabel(r"$\omega_n$")
        plt.ylabel(name + "[" + block + r"]$(i\omega_n)$")

    plt.tight_layout()
    plt.show()


## Deviations

We present a table containing deviations between the different solvers measured via

$$||G_{\rm Solver_1} - G_{\rm Solver_2}||_\infty$$

In [ ]:
import numpy as np

for block in block_lst:
    deviations = [[ np.amax(np.abs(G[s1][block].data - G[s2][block].data)) for s1 in solver_lst ] \
                    for s2 in solver_lst ]
    
    print("\t\t    Deviations for Block " + block)
    print("\t\t -----------------------------------")

    row_format ="{:>15}" * (len(solver_lst) + 1)
    print(row_format.format("", *solver_lst))
    row_format ="{:>15}" + "{:>15.2E}" * len(solver_lst)
    for solver, row in zip(solver_lst, deviations):
        print(row_format.format(solver, *row))
        
    print("\n\n")